In [1]:
import torch 
import numpy as np 
import pathlib
import json 

In [19]:

def load_questions_and_annotations(path):
    path = pathlib.Path(path) 
    question_path = path.joinpath("questions.json")
    annotation_path = path.joinpath("annotations.json")
    print(f"loading from {path}")
    with open(question_path) as qf, open(annotation_path) as af:
        questions = json.load(qf)
        annotations = json.load(af)
    return questions, annotations

def load_training_vectors(path):
    path = pathlib.Path(path)
    vectors = path.glob("*.pt")
    return vectors


def lookup_question_and_annotation(questions, annotations, question_id):
    for i, (quest, ann) in enumerate(zip(questions['questions'], annotations['annotations'])): 
        if str(quest['question_id']) == str(question_id):
            return quest, ann
    return -1, -1 



In [6]:
training_qs, training_anns = load_questions_and_annotations("/brtx/603-nvme2/estengel/annotator_uncertainty/vqa/train_anns") 
testing_qs, testing_anns = load_questions_and_annotations("/home/estengel/annotator_uncertainty/models/test_fixtures/data/real_vqa")

loading from /brtx/603-nvme2/estengel/annotator_uncertainty/vqa/train_anns
loading from /home/estengel/annotator_uncertainty/models/test_fixtures/data/real_vqa


In [7]:
from tqdm import tqdm 
best_idxs = []
best_sims = []

import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

train_paths = [x for x in load_training_vectors("/brtx/602-nvme2/estengel/annotator_uncertainty/vqa/precomputed_retrieval/")]
test_paths = [x for x in load_training_vectors("/home/estengel/annotator_uncertainty/models/test_fixtures/data/real_vqa/precomputed/")]

# train_subset = train_paths[400000:600000]
train_vectors = [torch.load(x, map_location="cpu").reshape(1, -1) for x in tqdm(train_subset)]
# test_vectors = [torch.load(x, map_location="cpu").reshape(1, -1) for x in test_paths]
# train_vectors = [torch.load(x, map_location="cpu").reshape(1, -1) for x in load_training_vectors("/brtx/602-nvme2/estengel/annotator_uncertainty/vqa/precomputed_retrieval/")]
# test_vectors = [torch.load(x, map_location="cpu").reshape(1, -1) for x in load_training_vectors("/home/estengel/annotator_uncertainty/models/test_fixtures/data/real_vqa/precomputed/")]

train_matrix = torch.cat(train_vectors, dim = 0)
torch.save(train_matrix, "/brtx/602-nvme2/estengel/annotator_uncertainty/vqa/precomputed_retrieval/shards/train_matrix_shard_2.pt")
# test_matrix = torch.cat(test_vectors, dim = 0)
# print(train_matrix.shape)
# print(test_matrix.shape)



# for i, vp in enumerate(test_vectors):
#     print(f"{i} of {len(test_vectors)}")
#     vec = torch.load(vp).reshape(-1)
#     best_sim = -1
#     best_idx = -1 

#     for j, tp in tqdm(enumerate(train_vectors), total=len(train_vectors)): 
#         tec = torch.load(tp).reshape(-1)
#         sim = torch.nn.CosineSimilarity(dim=0)(vec, tec)
#         if sim > best_sim:
#             best_sim = sim
#             best_idx = j
#     best_idxs.append(best_idx)
#     best_sims.append(best_sim)
    # print(vp, vec.shape)

100%|██████████| 43757/43757 [01:51<00:00, 392.20it/s]


In [8]:
print("shard 0")
shard0 = torch.load("/brtx/602-nvme2/estengel/annotator_uncertainty/vqa/precomputed_retrieval/shards/train_matrix_shard_0.pt")
print("shard 1")
shard1 = torch.load("/brtx/602-nvme2/estengel/annotator_uncertainty/vqa/precomputed_retrieval/shards/train_matrix_shard_1.pt")
print("shard 2")
shard2 = torch.load("/brtx/602-nvme2/estengel/annotator_uncertainty/vqa/precomputed_retrieval/shards/train_matrix_shard_2.pt")

shard 0
shard 1
shard 2


In [9]:
train_matrix = torch.cat([shard0, shard1, shard2], dim=0)

In [11]:
test_vectors = [torch.load(x, map_location="cpu").reshape(1, -1) for x in load_training_vectors("/home/estengel/annotator_uncertainty/models/test_fixtures/data/real_vqa/precomputed/")]
test_matrix = torch.cat(test_vectors, dim = 0)

In [12]:

import torch

def sim_matrix(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

sim = sim_matrix(test_matrix, train_matrix)



torch.Size([2, 443757])

In [21]:

filtered_training_qs, filtered_training_anns = load_questions_and_annotations("/brtx/603-nvme2/estengel/annotator_uncertainty/vqa/filtered")

top_idxs = sim.argmax(dim=-1)
test_closest_train_examples = [train_paths[i] for i in top_idxs]

for filename in test_closest_train_examples:
    base = pathlib.Path(filename).stem
    img_id, question_id, __ = base.split("_")
    question, annotation = lookup_question_and_annotation(filtered_training_qs, filtered_training_anns, question_id)
    print(question)
    print(annotation)

loading from /brtx/603-nvme2/estengel/annotator_uncertainty/vqa/filtered
-1
-1
{'image_id': 77055, 'question': 'What color is the bus?', 'question_id': 77055015}
{'answer_type': 'other', 'multiple_choice_answer': 'white', 'answers': [{'answer': 'white', 'answer_confidence': 'yes', 'answer_id': 1}, {'answer': 'black and white', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'white', 'answer_confidence': 'maybe', 'answer_id': 3}, {'answer': 'white', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'white', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'white', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'white and blue', 'answer_confidence': 'maybe', 'answer_id': 7}, {'answer': 'white and green', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'white', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'white', 'answer_confidence': 'yes', 'answer_id': 10}], 'image_id': 77055, 'question_type': 'what color is the', 'question_id': 77055015}